# Regression of Used Car Prices
1. Read this [data](https://www.kaggle.com/competitions/playground-series-s4e9/data)
2. Visualize this data by using Seaborn.
3. Handle **missing** and **outlier** data.
4. **Feature Engineering** -> Create **NewFeature** **Scaling**&**Encoding**
5. Build first model. This can be so simple.
6. Improve this first simple model by exploring the other method.
7. Evaluate the models.
8. **Hyperparameter Tuning**
9. **Model Ensemble**
10. **Prepare the Submission File**

In [13]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PyQt5.QtCore.QByteArray import length
from sklearn.linear_model import LinearRegression

In [6]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv('data/test.csv')

In [33]:
train_df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,other,Gray,0,NaN,4200
1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,1,NaN,4999
2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,0,NaN,13900
3,other,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,0,NaN,45000
4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,0,NaN,97500


In [31]:
test_df.head()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [11]:
train_df.drop(columns=['id'], inplace=True)
test_df.drop(columns=['id'], inplace=True)

KeyError: "['id'] not found in axis"

In [26]:
# 閾値
freq_threshold = 1000

# カテゴリ列
categorical_columns = ['brand', 'fuel_type', 'transmission', 'ext_col', 'int_col']

# 低頻度カテゴリを "other" に置き換える関数
def replace_low_frequency_categories(df, column, threshold):
    # カテゴリとその頻度を計算
    counts = df[column].value_counts()

    # 頻度が閾値未満のカテゴリを "other" に置き換える
    df[column] = df[column].apply(lambda x: x if counts.get(x, 0) >= threshold else 'other')

# 各カテゴリ列に対して処理を実行
for column in categorical_columns:
    replace_low_frequency_categories(train_df, column, freq_threshold)

In [38]:
for column in categorical_columns:
    replace_low_frequency_categories(test_df, column, freq_threshold)

In [32]:
train_df['clean_title'] = train_df['clean_title'].map({'Yes':0, 'NaN':1})

In [ ]:
test_df['clean_title'] = test_df['clean_title'].map({'Yes':0, 'NaN':1})

In [21]:
# 事故歴があるかどうか
train_df['accident'] = train_df['accident'].apply(lambda x: 0 if x == 'None reported' else 1)

In [39]:
test_df['accident'] = test_df['accident'].apply(lambda x: 0 if x == 'None reported' else 1)

In [35]:
train_df.drop(columns=['model'], axis=1, inplace=True)
train_df.drop(columns=['engine'], axis=1, inplace=True)

In [41]:
test_df.drop(columns=['model'], axis=1, inplace=True)
test_df.drop(columns=['engine'], axis=1, inplace=True)

In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_df[['model_year', 'milage']] = scaler.fit_transform(train_df[['model_year', 'milage']])

In [44]:
test_df[['model_year', 'milage']] = scaler.fit_transform(test_df[['model_year', 'milage']])

In [45]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)
model.fit(train_df.drop(columns=['price']), train_df['price'])

importances = model.feature_importances_
feature_importance = pd.DataFrame({'feature': train_df.drop(columns=['price']).columns, 'importance': importances})
print(feature_importance)

ValueError: could not convert string to float: 'MINI'